In [ ]:
import json
import os
import csv
import urllib
from io import BytesIO
from PIL import Image

from socket import timeout

In [ ]:
import torch
from torchvision import models
from torch.utils.data import Dataset, SubsetRandomSampler
from torchvision import transforms
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

device = torch.device("cuda:0")

In [ ]:
train_folder = "train_kaggle/"
print('Number of files in the train folder', len(os.listdir(train_folder)))

test_folder = "test_kaggle/"
print('Number of files in the test folder', len(os.listdir(test_folder)))

In [ ]:
class HotdogOrNotDataset(Dataset):
# Класс датасета для получения метки (хотдог/нет) по имени файла
    def __init__(self, folder, transform=None):
        self.transform = transform
        self.folder = folder
        
    def __len__(self):
        return(len(os.listdir(self.folder)))
    
    def __getitem__(self, index):
        y = 0
        filenames = os.listdir(self.folder)
        img_id = filenames[index]
        img_name = os.path.join(self.folder, img_id)
        if self.transform:
          img = self.transform(Image.open(img_name))
        else:
          img = Image.open(img_name)
        if ('frankfurter' in img_id) | ('chili-dog' in img_id) | ('hotdog' in img_id):
          y = 1
        return img, y, img_id

def visualize_samples(dataset, indices, title=None, count=10):
# Визуализация объектов датасета по индексам
    plt.figure(figsize=(count*3,3))
    display_indices = indices[:count]
    if title:
        plt.suptitle("%s %s/%s" % (title, len(display_indices), len(indices)))        
    for i, index in enumerate(display_indices):    
        x, y, _ = dataset[index]
        plt.subplot(1,count,i+1)
        plt.title("Label: %s" % y)
        plt.imshow(x)
        plt.grid(False)
        plt.axis('off')   
    
orig_dataset = HotdogOrNotDataset(train_folder)
indices = np.random.choice(np.arange(len(orig_dataset)), 7, replace=False)

visualize_samples(orig_dataset, indices, "Samples")

In [ ]:
train_dataset = HotdogOrNotDataset(train_folder, 
                       transform=transforms.Compose([
                           transforms.Resize((224, 224)),
                           transforms.ToTensor(),
                           transforms.RandomVerticalFlip(0.9),
                           transforms.ColorJitter(hue=.20, saturation=.20),
                           transforms.RandomRotation(10),
                           transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])                         
                       ])
                      )
test_dataset = HotdogOrNotDataset(test_folder, 
                       transform=transforms.Compose([
                           transforms.Resize((224, 224)),
                           transforms.ToTensor(),
                           transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])                         
                       ])
                      )

In [ ]:
batch_size = 16

data_size = len(orig_dataset)
validation_fraction = 0.2

indices = list(range(data_size))
train_indices, val_indices = train_test_split(indices, test_size=validation_fraction)

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
val_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                         sampler=val_sampler)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
def train_model(model, train_loader, val_loader, loss, optimizer, lr_scheduler, num_epochs):
# Тренировка модели НС с валидацией по эпохам    
    loss_history = []
    train_history = []
    val_history = []
    for epoch in range(num_epochs):
        model.train()
        loss_accum = 0
        correct_samples = 0
        total_samples = 0
        for i_step, (x, y, _) in enumerate(train_loader):
            x_gpu = x.to(device)
            y_gpu = y.to(device)
            prediction = model(x_gpu)    
            loss_value = loss(prediction, y_gpu)
            optimizer.zero_grad()
            loss_value.backward()
            optimizer.step()
            
            _, indices = torch.max(prediction, 1)
            correct_samples += torch.sum(indices == y_gpu)
            total_samples += y.shape[0]
            loss_accum += loss_value

        ave_loss = loss_accum / i_step
        train_accuracy = float(correct_samples) / total_samples
        val_accuracy = compute_accuracy(model, val_loader)
        
        lr_scheduler.step()
        loss_history.append(float(ave_loss))
        train_history.append(train_accuracy)
        val_history.append(val_accuracy)
        
        print("Average loss: %f, Train accuracy: %f, Val accuracy: %f" % (ave_loss, train_accuracy, val_accuracy))
    return loss_history, train_history, val_history
        
def compute_accuracy(model, loader):
# Расчет точности (accuracy) модели НС, возвращает точность в промежутке [0,1]    
    model.eval()

    correct_num = 0
    total_num = 0
    for i_step, (x, y,_) in enumerate(loader):
        x_gpu = x.to(device)
        y_gpu = y.to(device)
        prediction = model(x_gpu)
        indices = torch.argmax(prediction, 1)
        correct_num += torch.sum(indices == y_gpu)
        total_num += y.shape[0]

    accuracy = correct_num / total_num
    return accuracy

In [ ]:
from torch.utils.data.sampler import Sampler

class SubsetSampler(Sampler):
# Последовательная выдача индексов элементов датасета
    def __init__(self, indices):
        self.indices = indices

    def __iter__(self):
        return (self.indices[i] for i in range(len(self.indices)))

    def __len__(self):
        return len(self.indices)
    
    
def evaluate_model(model, dataset, indices):
# Расчет предсказания модели НС по индексам датасета, возвращает предсказание и истинную метку
    model.eval() # Evaluation mode
    
    predictions = np.array([], int)
    ground_truth = np.array([], int)
    sampler = SubsetSampler(indices)
    data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         sampler=sampler)
    for i_step, (x, y, _) in enumerate(data_loader):
        x_gpu = x.to(device)
        y_gpu = y.to(device)
        model_out = model(x_gpu)
        pred = torch.argmax(model_out, 1)
        predictions=np.append(predictions, pred.to('cpu'))
        ground_truth=np.append(ground_truth, y)
    return predictions, ground_truth

In [ ]:
import sklearn.metrics as metrics
def binary_classification_metrics(prediction, ground_truth):
# Расчет precision, recall, f1-меры
    precision, recall, f1, _ = metrics.precision_recall_fscore_support(prediction, ground_truth, average='binary')
    return precision, recall, f1

In [ ]:
%%time
import torch.nn as nn
import torch.optim as optim
from collections import namedtuple

batch_size = 16
lr_feat = 1e-4
lr_out = 1e-3
anneal_coeff = 0.5
anneal_epoch = 1

    

model = models.convnext_tiny(weights='DEFAULT')
num_ftrs = model.classifier[2].in_features
model.classifier[2] = nn.Linear(in_features=num_ftrs, out_features=2)
model.to(device)
train_dataset = HotdogOrNotDataset(train_folder, 
                                            transform=transforms.Compose([
                                                transforms.Resize((224, 224)),
                                                transforms.ToTensor(),
                                                transforms.RandomHorizontalFlip(0.2),
                                                transforms.RandomVerticalFlip(0.2),
                                                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                        std=[0.229, 0.224, 0.225])                         
                                        ])
                                    )
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler)
val_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=val_sampler)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

loss = nn.CrossEntropyLoss()

output_params = list(map(id, model.classifier.parameters()))
feature_params = filter(lambda p: id(p) not in output_params, model.parameters())
optimizer = optim.Adam([{'params': model.classifier.parameters()},
                                                   {'params': feature_params, 'lr': lr_feat}],
                                                   lr = lr_out)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=anneal_epoch, gamma=anneal_coeff)
loss_history, train_history, val_history = train_model(model, train_loader,
                                                                                   val_loader, loss, optimizer, lr_scheduler, 3)                           

In [ ]:
predictions, ground_truth = evaluate_model(model, train_dataset, val_indices)
precision, recall, f1 = binary_classification_metrics(predictions, ground_truth)
print("F1: %f, P: %f, R: %f" % (f1, precision, recall))

In [ ]:
false_positive_indices = np.array(val_indices)[(ground_truth==0)&(predictions==1)]
visualize_samples(orig_dataset, false_positive_indices, "False positives")

false_negatives_indices = np.array(val_indices)[(ground_truth==1)&(predictions==0)]
visualize_samples(orig_dataset, false_negatives_indices, "False negatives")

In [ ]:
image_id = []
predictions = np.array([], int)
model.eval()
model.to(device)
for x,_,id_img in test_loader:
    model_out = model(x.to(device))
    pred = torch.argmax(model_out, 1)
    predictions=np.append(predictions, pred.to('cpu'))
    image_id=np.append(image_id, id_img)

In [ ]:
with open('subm_torch.csv', 'w') as submissionFile:
    writer = csv.writer(submissionFile)
    writer.writerow(['image_id', 'label'])
    writer.writerows(zip(image_id,predictions))

In [ ]:
torch.save(model, 'hotdogTorch_model.pth')